## Criação do DataSet

In [ ]:
!pip install albumentations==0.4.6
import torch
import numpy as np
from skimage.transform import resize
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
import random
import os
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt



     |████████████████████████████████| 117 kB 4.9 MB/s 
     |████████████████████████████████| 948 kB 9.6 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=3c9be597e0f527a1b990340467be0b2c775d1020d2bcd44ed2e72be8f90aeca6
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#######################################################
#               Define Transforms
#######################################################

train_transforms = A.Compose(
    [
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=360, p=0.5),
        A.Resize(224,224),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5), 
        ToTensorV2(),
      
    ]
)

test_transforms = A.Compose(
    [
        A.Resize(224,224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]
)

In [ ]:
####################################################
#       Create Train, Valid and Test sets
####################################################

#Eduardo_drive (fonte)
# train_data_path = '/content/drive/MyDrive/Colab Notebooks/equipes_f1_train' 
# test_data_path =  '/content/drive/MyDrive/Colab Notebooks/equipes_f1_test' 

# #Ricardo_drive (atalho)
# caminho_0 = '/content/drive/MyDrive/Raiz/Faculdade/2021.2/Deep Learning/{}'
# test_data_path = caminho_0.format('equipes_f1_test_1') 
# train_data_path =  caminho_0.format('equipes_f1_train_1')  

# #Ricardo_2_drive (atalho)
caminho_1 = '/content/drive/MyDrive/@ufg.br/Faculdade/2021.2/Deep Learning/{}'
test_data_path = caminho_1.format('equipes_f1_test_1') 
train_data_path =  caminho_1.format('equipes_f1_train_1')  

train_image_paths = [] #armazena os caminhos em uma lista
classes = [] #para armazenar os valores das classes

for data_path in glob.glob(train_data_path + '/*'):
    classes.append(data_path.split('/')[-1]) 
    train_image_paths.append(glob.glob(data_path + '/*'))
    
train_image_paths = [item for sublist in train_image_paths for item in sublist]
train_image_paths = list(train_image_paths )
random.shuffle(train_image_paths)

print('train_image_path example: ', train_image_paths[0])
print('class example: ', classes[0])

#2.
# split train valid de train paths (80,20)
train_image_paths, valid_image_paths = train_image_paths[:int(0.8*len(train_image_paths))], train_image_paths[int(0.8*len(train_image_paths)):] 

#3.
# cria test_image_paths
test_image_paths = []
for data_path in glob.glob(test_data_path + '/*'):
    test_image_paths.append(glob.glob(data_path + '/*'))

test_image_paths = [item for sublist in test_image_paths for item in sublist]
test_image_paths = list(test_image_paths )

print("Train size: {}\nValid size: {}\nTest size: {}".format(len(train_image_paths), len(valid_image_paths), len(test_image_paths)))


train_image_path example:  /content/drive/MyDrive/Colab Notebooks/equipes_f1_train/williams/williams (25).jpg
class example:  williams
Train size: 128
Valid size: 32
Test size: 100


In [ ]:
test_data_path

'/content/drive/MyDrive/@ufg.br/Faculdade/2021.2/Deep Learning/equipes_f1_test_1'

In [ ]:
#######################################################
#      Create dictionary for class indexes
#######################################################

idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

In [ ]:
idx_to_class

{0: 'williams',
 1: 'ferrari',
 2: 'alpine',
 3: 'alphatauri',
 4: 'redbull',
 5: 'astonmartin',
 6: 'haas',
 7: 'alpharomeo',
 8: 'mercedes',
 9: 'mclaren'}

In [ ]:
#######################################################
#               Define Dataset Class
#######################################################

class LandmarkDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label    



In [ ]:
#######################################################
#                  Create Dataset
#######################################################

train_dataset = LandmarkDataset(train_image_paths,train_transforms)
valid_dataset = LandmarkDataset(valid_image_paths,test_transforms) #test transforms are applied
test_dataset = LandmarkDataset(test_image_paths,test_transforms)

In [ ]:
print('The shape of tensor for 50th image in train dataset: ',train_dataset[49][0].shape)
print('The label for 50th image in train dataset: ',train_dataset[49][1])

The shape of tensor for 50th image in train dataset:  torch.Size([3, 224, 224])
The label for 50th image in train dataset:  0


In [ ]:
import matplotlib.pyplot as plt
import copy
%matplotlib inline

#######################################################
#                  Visualize Dataset
#         Images are plotted after augmentation
#######################################################

def visualize_augmentations(dataset, idx=0, samples=20, cols=10, random_img = False):
    
    dataset = copy.deepcopy(dataset)
    #we remove the normalize and tensor conversion from our augmentation pipeline
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    
        
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(24, 6))
    for i in range(samples):
        if random_img:
            idx = np.random.randint(1,len(train_image_paths))
        image, lab = dataset[idx]
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
        ax.ravel()[i].set_title(idx_to_class[lab])
    plt.tight_layout(pad=1)
    plt.show()    

visualize_augmentations(train_dataset,np.random.randint(1,len(train_image_paths)), random_img = True)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from torch.utils.data import DataLoader


In [ ]:
#######################################################
#                  Define Dataloaders
#######################################################
batch_size = 10

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

valid_loader = DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)

In [ ]:
import torchvision
from torchvision import datasets, models, transforms


#importando modelo vgg 19
#vgg19 = models.vgg19(pretrained=True)
vgg19_f = models.vgg19(pretrained=False)
# print(vgg19_f)
vgg19 = vgg19_f
print(vgg19)
# resnet = models.resnet34(pretrained=True)
# print(resnet)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

## Modelo e treinamento

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F

# #########################################
# ##################  MODELO 4  ###########
# #########################################

# initial_input = 224
# h = 512*7*7
# header_2 = int((h)/4)
# header_3 = int((h)/8)
# header_4 = int((h)/16)
# header_5 = int((h)/32)

# out_channel_1 = 64
# out_channel_2 = 128
# out_channel_3 = 512
# out_channel_4 = 3584

# class Net_4(nn.Module):
#   #baseado na arquitetura do vgg19
#     def __init__(self):
#         super(Net_4, self).__init__()
#         # convolutional layer

#         self.conv1 = nn.Conv2d(3, out_channel_1, 2, padding=1) # 64x224x224 -> Size_output
#         self.conv2 = nn.Conv2d(out_channel_1,out_channel_1,3 ,padding=1) # 64x224x224 -> Size_output
        
#         #pool_/2 ->32x112x112
#         self.conv3 = nn.Conv2d(out_channel_1,out_channel_2,2, padding=1) #128x112x112 -> Size_output
#         self.conv4 = nn.Conv2d(out_channel_2,out_channel_2,2,padding=1) #128x112x112-> Size_output

#         #pool_/4 -> 256x28x28
#         self.conv5 = nn.Conv2d(out_channel_2,out_channel_3,2,padding=1) #512x28x28
#         self.conv6 = nn.Conv2d(out_channel_3,out_channel_3,2,padding=1) #512x28x28
#         self.conv7 = nn.Conv2d(out_channel_3,out_channel_3,2,padding=1) #512x28x28
#         self.conv8 = nn.Conv2d(out_channel_3,out_channel_3,2,padding=1) #512x28x28
#         self.conv9 = nn.Conv2d(out_channel_3,out_channel_3,2)  #512x7x7

#         # max pooling layer
#         '''
#         As dimensionalidades do maxpool foram pensadas buscando atender há:
#           1.Diminuir a dimensionalidade da rede sem perder sua complexidade
#          '''
#         self.pool_1 = nn.MaxPool2d(2, 2) 
#         self.pool_2 = nn.MaxPool2d(4,4)
#         self.pool_3 = nn.MaxPool2d(7,7)

#         #Fully-conected-layer
#         self.c1 = nn.Linear(h,header_2)
#         # self.c2 = nn.Linear(header_2,header_3)
#         # self.c3 = nn.Linear(header_3,header_4)
#         # self.c4 = nn.Linear(header_4,header_5)
#         self.c5 = nn.Linear(header_2,10)


#         self.dropout = nn.Dropout(0.2)
#     def forward(self, x):
#         # add sequence of convolutional and max pooling layers
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
        
#         #pool_/2
#         x = self.pool_1(x)
#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
#         x = F.relu(self.conv5(x))
#         x = F.relu(self.conv6(x))

#         #pool_/4
#         x = self.pool_2(x)
#         x = F.relu(self.conv7(x))
#         x = F.relu(self.conv8(x))
#         x = F.relu(self.conv9(x))
#         x = self.pool_2(x)
    
        
#         x = x.view(-1,int(h)) #reshape input
        
#         x = self.dropout(F.relu(self.c1(x)))
#         x = self.dropout((self.c5(x)))

#         return x
# model = Net_4()
# print(model)
# if train_on_gpu:
#     model.cuda()



In [ ]:

import torch.nn as nn
import torch.nn.functional as F

initial_input = 224
h = 512*7*7
header_2 = int((h)/4)
header_3 = int((h)/8)
header_4 = int((h)/16)
header_5 = int((h)/32)

out_channel_1 = 64
out_channel_2 = 128
out_channel_3 = 512

class Cnn_Net(nn.Module):
    def __init__(self):
        super(Cnn_Net,self).__init__()
            
        #Convolutional Layer
        self.conv0 = nn.Conv2d(3,out_channel_1,3,padding=1) # -> 224x224x64
        self.conv1 = nn.Conv2d(out_channel_1,out_channel_1,3,padding=1) # -> 224x224x64
        self.conv2 = nn.Conv2d(out_channel_1,out_channel_1,3,padding=1) # -> 224x224x64
        
        #Poll/2 -> 112x112x64
        self.conv3 = nn.Conv2d(out_channel_1,out_channel_2,3,padding=1) # ->112x112x128
        self.conv4 = nn.Conv2d(out_channel_2,out_channel_2,3,padding=1) # -> 112x112x128
       
        #Pool/4 -> 28x28x128
        self.conv5 = nn.Conv2d(out_channel_2,out_channel_3,3,padding=1) # -> 112x112x128
        self.conv6 = nn.Conv2d(out_channel_3,out_channel_3,3,padding=1) # -> 28x28x512
        self.conv7 = nn.Conv2d(out_channel_3,out_channel_3,3,padding=1) # -> 28x28x512
        self.conv8 = nn.Conv2d(out_channel_3,out_channel_3,3,padding=1) # -> 28x28x512
        #Pool/4 -> 7x7x512

        #Max_Pool
        self.pool1 = nn.MaxPool2d(2,2)
        self.pool2 = nn.MaxPool2d(4,4)

        #Batch_normalize
        self.batch_norm1 = nn.BatchNorm2d(out_channel_1)
        self.batch_norm2 = nn.BatchNorm2d(out_channel_2) 
        self.batch_norm3 = nn.BatchNorm2d(out_channel_3)

        #Fully_conected_layer
        self.c1 = nn.Linear(h,10)
        
        
        self.dropout = nn.Dropout(0.25)

            
    def forward(self,x):
        #Agora são implementadas as "shortcut connections", utilizadas para
        #tratar o gradiente de fuga
        x = F.relu(self.batch_norm1(self.conv0(x)))
        x1 = x
        x = F.relu(self.batch_norm1(self.conv1(x)))
        x = (self.batch_norm1(self.conv2(x)))
        x += x1
        x = F.relu(x)
        x = self.pool1(x)
        
        
        x = self.dropout(x)
        x = F.relu(self.batch_norm2(self.conv3(x)))
        x2 = x
        x = F.relu(self.batch_norm2(self.conv4(x)))
        x = self.batch_norm2(self.conv4(x))
        #x += x2
        x = F.relu(x)
        x = self.pool2(x)
        x = self.dropout(x)
        #x3 = x
        x = F.relu(self.batch_norm3(self.conv5(x)))
        x3 = x
        x = F.relu(self.batch_norm3(self.conv6(x)))
        x = self.batch_norm3(self.conv7(x))
        x += x3
        x = F.relu(x)
        x = self.dropout(x)
       
        x = F.relu(self.batch_norm3(self.conv7(x)))
        x4 = x
        x = F.relu(self.batch_norm3(self.conv8(x)))
        x = self.batch_norm3(self.conv8(x))
        x += x4
        x = F.relu(x)
        x=self.pool2(x)
        x = self.dropout(x)
        
        #Fully-connected_layer
        x = x.view(-1,h)
        x = self.c1(x)
       
        
        return x
    
model = Cnn_Net()
print(model)
if train_on_gpu:
    model.cuda()

Cnn_Net(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): Ba

In [ ]:
!nvidia-smi

Mon Feb 28 18:40:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    75W / 149W |    559MiB / 11441MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Verificando quantidade de parametros no modelo
initial_input = 224
header_1 = (((initial_input/(2**4))**2) * 256)
header_2 = ((header_1)/4)
header_3 = ((header_2)/8)

print('1: {}, 2: {}, 1: {}'.format(header_1,header_2,header_3))

1: 50176.0, 2: 12544.0, 1: 1568.0


In [ ]:
#loss_function
import torch
criterion = torch.nn.CrossEntropyLoss()

#optimizer function
lr = 0.01
# optimizer = torch.optim.SGD(vgg19.classifier.parameters(), lr = lr)
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [ ]:
modelo = 'nsew_model'
import numpy as np
# number of epochs to train the model
n_epochs = 50 # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    if epoch == 0:
      epoch_last_saved = 0
    if (epoch_last_saved == (epoch - 15)): #caso esteja na décima época, ou, há 10 épocas em que não é identificado um mínimo error:
        lr = lr/2
        optimizer = torch.optim.SGD(model.parameters(), lr = lr) #Diminui o "passo" do gradiente
        print('Atualizando o Lr para {}'.format(lr))
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        # torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/models/{}.pt'.format(modelo))
        torch.save(model.state_dict(), '/content/drive/MyDrive/@ufg.br/Faculdade/2021.2/Deep Learning//Models/{}.pt'.format(modelo))
        epoch_last_saved = epoch
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 2.367160 	Validation Loss: 6.240395
Validation loss decreased (inf --> 6.240395).  Saving model ...
Epoch: 2 	Training Loss: 2.335060 	Validation Loss: 5.688807
Validation loss decreased (6.240395 --> 5.688807).  Saving model ...
Epoch: 3 	Training Loss: 2.324151 	Validation Loss: 6.400615
Epoch: 4 	Training Loss: 2.289753 	Validation Loss: 12.900865
Epoch: 5 	Training Loss: 2.288098 	Validation Loss: 7.568432
Epoch: 6 	Training Loss: 2.297046 	Validation Loss: 4.146555
Validation loss decreased (5.688807 --> 4.146555).  Saving model ...
Epoch: 7 	Training Loss: 2.295787 	Validation Loss: 4.545342


KeyboardInterrupt: ignored

In [ ]:
# model.cpu()

modelo = 'new_model'
model.load_state_dict(torch.load('./models/{}.pt'.format(modelo),map_location=torch.device('cpu')))
# torch.save(model.state_dict(), '/content/drive/MyDrive/Raiz/Faculdade/2021.2/Deep Learning/Models/{}.pt'.format(modelo))

### Testando model_1

In [ ]:
# track test loss
def test_model(model):
  test_loss = 0.0
  class_correct = list(0. for i in range(10))
  class_total = list(0. for i in range(10))

  model.eval()
  # iterate over test data
  for data, target in test_loader:
      # move tensors to GPU if CUDA is available
      if train_on_gpu:
          data, target = data.cuda(), target.cuda()
      # forward pass: compute predicted outputs by passing inputs to the model
      output = model(data)
      # calculate the batch loss
      loss = criterion(output, target)
      # update test loss 
      test_loss += loss.item()*data.size(0)
      # convert output probabilities to predicted class
      _, pred = torch.max(output, 1)    
      # compare predictions to true label
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
      # calculate test accuracy for each object class
      for i in range(batch_size):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1
 
  # average test loss
  test_loss = test_loss/len(test_loader.dataset)
  print('Test Loss: {:.6f}\n'.format(test_loss))

  for i in range(10):
      if class_total[i] > 0:
          print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
      else:
          print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

  print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
      100. * np.sum(class_correct) / np.sum(class_total),
      np.sum(class_correct), np.sum(class_total)))

test_model(model)


## Confusion Matrix

In [ ]:
model.cpu()

In [ ]:
def createCM(loader,model,nClasses):
    from sklearn.metrics import confusion_matrix
    accuracy = 0
    CM = np.zeros((nClasses,nClasses))
    y_pred = []
    y_real = []
    #COLOQUE O SEU CÓDIGO AQUI
    with torch.no_grad():
       for inputs, targets in loader:
          outputs = model(inputs)
          y_pred.extend((torch.max(outputs,1)[1]).numpy())
          y_real.extend(targets.numpy())
    CM = confusion_matrix(y_real,y_pred)
    values_correctly = 0
    for l in range(0,nClasses):
        values_correctly += CM[l][l]
    accuracy = ((100/CM.sum()) * values_correctly)
    #Para cada classe  procura os valores do tensor que sao dessa classe, posteriormente compara com o 




    return CM, accuracy

In [ ]:
#NÃO ALTERE ESTA CÉLULA
def plot_CM(CM,accuracy):
    CM = 100*CM/CM.sum(axis=1)
    plt.figure(figsize=(10,8))
    plt.imshow(CM,cmap='YlOrBr')
    plt.ylabel('target')
    plt.xlabel('prediction')
    plt.title('Accuracy: '+"{:4.2f}".format(accuracy)+'%')
    plt.colorbar()
    for i in range(CM.shape[0]):
        for j in range(CM.shape[1]):
            if CM[i,j]>70:
                color = "White"
            else:
                color = "Black"
            plt.text(j, i, "{:.3f}".format(CM[i,j]), ha="center", va="center", color=color)    

CM_train, accuracy_train = createCM(train_loader, model, 10)
plot_CM(CM_train, accuracy_train)

In [ ]:
CM_train, accuracy_train = createCM(test_loader, model, 10)
plot_CM(CM_train, accuracy_train)

In [ ]:
CM_train, accuracy_train = createCM(valid_loader, model, 10)
plot_CM(CM_train, accuracy_train)